In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
#df=pd.read_csv("https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/DPQMQH/P2Z4PM")

In [2]:
df=pd.read_csv("D:\\Data fro internship\\news.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)

In [32]:
df2=pd.read_csv("D:\\Data fro internship\\stocks.csv")

In [7]:
df.head()

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
2,20010102,unknown,Fissures in Hurriyat over Pak visit
3,20010102,unknown,America's unwanted heading for India?
4,20010102,unknown,For bigwigs; it is destination Goa


In [8]:
def date_converter(date):
    year= int(str(date)[0:4])
    month=int(str(date)[4:6])
    day=int(str(date)[6:])
    return datetime.date(year,month,day)

In [9]:
df["publish_date"]=df["publish_date"].apply(date_converter)

In [10]:
df["headline_category"].value_counts()

india                      285619
unknown                    207732
city.mumbai                132649
city.delhi                 124658
business.india-business    115246
                            ...  
actresses                       7
delhi-ncr                       7
did-you-know                    6
sports.headline6                3
party-manifestos                2
Name: headline_category, Length: 1016, dtype: int64

In [11]:
def join_news(news):
    if news[0]=="unknown":
        return news[1]
    else:
        return " ".join([news[0],news[1]])

In [12]:
df["headline_content"]=df[["headline_category","headline_text"]].apply(lambda x:join_news(x),axis=1)

In [13]:
df.drop(["headline_category","headline_text"],axis=1,inplace=True)

In [14]:
df.head()

,publish_date,headline_content
0,2001-01-01,sports.wwe win over cena satisfying but defeat...
1,2001-01-02,Status quo will not be disturbed at Ayodhya; s...
2,2001-01-02,Fissures in Hurriyat over Pak visit
3,2001-01-02,America's unwanted heading for India?
4,2001-01-02,For bigwigs; it is destination Goa


In [43]:
df.groupby("publish_date").count()

,headline_content
publish_date,
2001-01-01,1
2001-01-02,86
2001-01-03,41
2001-01-04,153
2001-01-05,126
...,...
2020-06-26,500
2020-06-27,527
2020-06-28,500


In [44]:
X=df.groupby("publish_date").agg({
    'headline_content': lambda x: ' '.join(x)})

In [48]:
X.reset_index(inplace=True)

In [49]:
X.head()

,publish_date,headline_content
0,2001-01-01,sports.wwe win over cena satisfying but defeat...
1,2001-01-02,Status quo will not be disturbed at Ayodhya; s...
2,2001-01-03,Powerless north India gropes in the dark Think...
3,2001-01-04,The string that pulled Stephen Hawking to Indi...
4,2001-01-05,Light combat craft takes India into club class...


### for merging the output data

In [33]:
df2.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2001-01-02,4028.570068,3929.370117,3953.219971,4018.879883,0.0,4018.879883
1,2001-01-03,4067.659912,3977.580078,3977.580078,4060.020020,0.0,4060.020020
2,2001-01-04,4180.970215,4109.549805,4180.970215,4115.370117,0.0,4115.370117
3,2001-01-05,4195.009766,4115.350098,4116.339844,4183.729980,0.0,4183.729980
4,2001-01-08,4206.720215,4101.529785,4164.759766,4120.430176,0.0,4120.430176


In [34]:
df2["daily_return"]=df2["Close"].pct_change()

In [35]:
df2.drop(df2.drop(['daily_return','Date'],axis=1).columns,axis=1,inplace=True)

In [36]:
for i in df2.drop('daily_return',axis=1).columns:
    df2[i]=df2[i].shift()

In [56]:
df2.head()

,publish_date,daily_return
1,2001-01-02,0.010237
2,2001-01-03,0.013633
3,2001-01-04,0.016611
4,2001-01-05,-0.015130
5,2001-01-08,0.001184


In [53]:
df2.rename(columns={"Date":"publish_date"},inplace=True)

In [39]:
df2.drop(0,axis=0,inplace=True)

In [66]:
def to_date(date):
    return pd.to_datetime(date).date()

In [67]:
df2["publish_date"]=df2["publish_date"].apply(to_date)

In [68]:
X.head()

,publish_date,headline_content
0,2001-01-01,sports.wwe win over cena satisfying but defeat...
1,2001-01-02,Status quo will not be disturbed at Ayodhya; s...
2,2001-01-03,Powerless north India gropes in the dark Think...
3,2001-01-04,The string that pulled Stephen Hawking to Indi...
4,2001-01-05,Light combat craft takes India into club class...


In [72]:
df3=pd.merge(X,df2,on="publish_date")

In [74]:
df3.drop("publish_date",axis=1,inplace=True)

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

In [84]:
import re
from nltk.corpus import stopwords

In [85]:
stop_words = set(stopwords.words('english'))
      
def preprocess_text(txt):
    txt = re.sub('[^a-zA-Z0-9 ,.]','',txt) #removing all characters which are not useful other than alphabets,blank space,comma and puctuation.
    tokens = txt.split()
    tokens = [w for w in tokens if w.lower() not in stop_words] #since we are focusing on important keywords that determine the catagory not the other features .we are removing stopwords
    txt = ' '.join(tokens)
    return txt

In [86]:
df3["headline_content"] = df3["headline_content"].apply(lambda x : preprocess_text(x))

In [87]:
y=df3["daily_return"]
X=df3["headline_content"]

In [96]:
def y_change(y):
    if y>0:
        return 1
    else:
        return 0

In [97]:
y=y.apply(y_change)

In [88]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [90]:
X.shape

(4767, 239424)

In [92]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix,auc,roc_auc_score,classification_report
from sklearn.model_selection import cross_val_score,KFold

In [98]:
#with default parameters
lr=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=10)
nb=MultinomialNB()
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
bg=BaggingClassifier(dt)
# ab=AdaBoostClassifier()
# gb=GradientBoostingClassifier()

In [99]:
models = []
models.append(('LogisticRegression',lr))
models.append(('KNeighborsClassifier',knn))
models.append(('MultinomialNB',nb))
models.append(('DecisionTreeClassifier',dt))
models.append(('RandomForestClassifier',rf))
models.append(('BaggingClassifier',bg))
# models.append(('AdaBoostClassifier',ab))
# models.append(('GradientBoostingClassifier',gb))

In [100]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=5,shuffle=True,random_state=0)
    auc =cross_val_score(model,X,y,cv=kfold,scoring='f1_weighted')
    results.append(auc)
    names.append(name)
    print('%s: %f' % (name,np.mean(auc*100)))
    BE =1-np.mean(auc)#Bias error
    VE =np.var(1-auc,ddof=1)#variance error
    print("\t\tBais error : ", BE)
    print("\t\tVariance error: ",VE)
    
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.xticks(rotation=15)
plt.show()

C:\Users\brvis\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\brvis\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

LogisticRegression: 51.436978
		Bais error :  0.4856302198543492
		Variance error:  5.809089930655415e-05
KNeighborsClassifier: 49.818834
		Bais error :  0.5018116579851244
		Variance error:  0.0001162257278915307
MultinomialNB: 52.133556
		Bais error :  0.47866444005789
		Variance error:  0.0001477112370627805
DecisionTreeClassifier: 50.522940
		Bais error :  0.49477060242608883
		Variance error:  0.00023533681449945892
RandomForestClassifier: 50.216739
		Bais error :  0.4978326071085363
		Variance error:  0.00012589057682368032
BaggingClassifier: 50.534637
		Bais error :  0.4946536296524092
		Variance error:  0.00045313998239063995


NameError: name 'plt' is not defined